In [1]:
%pwd

'e:\\Data Science\\LoanApprovalAppWith-Mlflow\\Loan-APP\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'e:\\Data Science\\LoanApprovalAppWith-Mlflow\\Loan-APP'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    oob_score: bool
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    max_samples: float
    max_leaf_nodes: int
    max_depth: int
    max_features: str
    class_weight: str
    random_state: int

In [11]:
from Loan_APP.constants import *
from Loan_APP.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            target_column= schema.name,
            oob_score= params.oob_score,
            n_estimators= params.n_estimators,
            min_samples_split=params.min_samples_split,
            min_samples_leaf= params.min_samples_leaf,
            max_samples= params.max_samples,
            max_leaf_nodes= params.max_leaf_nodes,
            max_depth= params.max_depth,
            max_features= params.max_features,
            class_weight= params.class_weight,
            random_state= params.random_state
        )
        
        return model_trainer_config
        

In [13]:
import pandas as pd
import os
from Loan_APP import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [14]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = train_data[[self.config.target_column]]
        
        model = RandomForestClassifier(n_estimators= self.config.n_estimators, min_samples_split= self.config.min_samples_split, min_samples_leaf= self.config.min_samples_split, max_samples= self.config.max_samples, max_leaf_nodes= self.config.max_leaf_nodes, max_depth= self.config.max_depth, max_features= self.config.max_features, class_weight= self.config.class_weight, oob_score= self.config.oob_score, random_state= self.config.random_state)
        model.fit(train_x, train_y)
        
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        

In [15]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config= model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-03-12 21:59:48,869: INFO: common: yaml file: config\config.yaml is loaded successfully!]
[2025-03-12 21:59:48,880: INFO: common: yaml file: params.yaml is loaded successfully!]
[2025-03-12 21:59:48,886: INFO: common: yaml file: schema.yaml is loaded successfully!]
[2025-03-12 21:59:48,891: INFO: common: created directory at: artifacts]
[2025-03-12 21:59:48,896: INFO: common: created directory at: artifacts/model_trainer]


e:\Data Science\LoanApprovalAppWith-Mlflow\Loan-APP\env\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
